<a href="https://colab.research.google.com/github/mojtabaSefidi/Fall-Detection-System/blob/main/Fall_Detection_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries and frameworks

In [1]:
from sklearn import preprocessing
import tensorflow as tf
from math import sqrt
import pandas as pd
import numpy as np
import glob
import os
window_size = 200


Get the sisfall and sisfall_enhanced dataset

In [ ]:
!gdown --id 1kyTRhIFhqwRkf9gERof1Xm5FVQ-klLVA
!gdown --id 1gvOuxPc8dNgTnxuvPcVuCKifOf98-TV0
!unzip SisFall_dataset.zip
!unzip SisFall_enhanced.zip